# SkopeRules

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve
from skrules import SkopeRules
from preprocessing import *
import matplotlib.pyplot as plt 
import numpy as np

HAHAHA /Users/ankelovset/Documents/Master/2. Semester/INFO381-AI/info381


In [5]:
# Load the dataset 
data = clean_data("fraud.csv")

Importing data...
splitting time columns...
Finding natural clusters...


/Users/ankelovset/anaconda3/envs/skoperules38/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


adding distances
Dropping columns...
Rounding columns...
Index(['cc_num', 'merchant', 'category', 'amt', 'gender', 'street', 'city',
       'state', 'zip', 'city_pop', 'job', 'trans_num', 'is_fraud',
       'trans_minute', 'trans_hour', 'trans_day', 'trans_month', 'trans_year',
       'trans_dayofweek', 'dob_day', 'dob_month', 'dob_year',
       'merchant_buyer_distance', 'merchant_cluster_id', 'buyer_cluster_id'],
      dtype='object')
Encoding categorical features...
Rounding categorical columns...
             cc_num  merchant  category     amt  gender  street  city  state  \
0  2703186189652095       514         8    4.97       0     576   532     27   
1      630423337322       241         4  107.23       0     439   619     47   
2    38859492057661       390         0  220.11       1     610   474     13   
3  3534093764340240       360         2   45.00       1     945    84     26   
4   375534208663984       297         9   41.96       1     422   217     45   

     zip  cit

In [6]:
# Splitting features and target variable
X = data.drop("is_fraud", axis=1)
y = data["is_fraud"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [7]:
# Step 3: Fit the SkopeRules model
skope = SkopeRules(
    feature_names=X_train.columns,
    precision_min=0.5,
    recall_min=0.01,
    n_estimators=30,
    random_state=42
)

print("fitting the model")
skope.fit(X_train, y_train)

fitting the model


/Users/ankelovset/anaconda3/envs/skoperules38/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/ankelovset/anaconda3/envs/skoperules38/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


SkopeRules(feature_names=Index(['cc_num', 'merchant', 'category', 'amt', 'gender', 'street', 'city',
       'state', 'zip', 'city_pop', 'job', 'trans_num', 'trans_minute',
       'trans_hour', 'trans_day', 'trans_month', 'trans_year',
       'trans_dayofweek', 'dob_day', 'dob_month', 'dob_year',
       'merchant_buyer_distance', 'merchant_cluster_id', 'buyer_cluster_id'],
      dtype='object'),
           n_estimators=30, random_state=42)

In [8]:
#def predict_as_proba(X):
#    preds = skope.predict(X)  # skope.predict returns 0 or 1
#    return np.vstack([1 - preds, preds]).T  # Turn into probabilities

In [9]:
# Pick some explanation baseline 
#X_explain = X_train.sample(100, random_state=42)

# Use shap.Explainer (not KernelExplainer)
#explainer = shap.KernelExplainer(predict_as_proba, X_explain)

# Compute SHAP values
#X_test_sample = X_test.sample(100, random_state=42)
#shap_values = explainer.shap_values(X_test)

#shap.summary_plot(shap_values[1], X_test_sample, plot_type="violin", show=False)
#plt.savefig("shap_summary_violin2.png")
#plt.close()

In [10]:
print("finding rules for is_fraud")
rules = skope.rules_[:10]

print("printing rules")
for rule in rules:
    print(rule)
    print()
    print(20*'=')
    print()

finding rules for is_fraud
printing rules
('trans_hour > 21.5 and amt <= 1258.719970703125 and amt > 713.010009765625', (0.8612975391498882, 0.3246205733558179, 2))


('trans_hour > 21.5 and amt <= 1258.5549926757812 and amt > 712.3450012207031', (0.8526077097505669, 0.32164242942686055, 2))


('trans_hour > 21.5 and amt <= 1206.8699951171875 and amt > 682.2449951171875', (0.8150537634408602, 0.32728842832469773, 2))


('trans_hour > 21.5 and amt <= 1207.875 and amt > 713.0899963378906', (0.8554216867469879, 0.3178155774395703, 2))


('trans_hour > 21.5 and amt <= 1254.7449951171875 and amt > 713.7649841308594', (0.8785714285714286, 0.31297709923664124, 2))


('trans_hour > 21.5 and amt <= 1260.6849975585938 and amt > 713.0899963378906', (0.8554502369668247, 0.31013745704467355, 2))


('trans_hour > 21.5 and amt <= 1260.6849975585938 and amt > 693.5050048828125', (0.8422273781902552, 0.3107876712328767, 2))


('trans_hour > 21.5 and amt <= 1260.6849975585938 and amt > 713.6799926757812

In [11]:
# Step 4: Predict and evaluate
y_pred_skope = skope.predict(X_test)

print("SkopeRules Classification Report:")
print(classification_report(y_test, y_pred_skope))

SkopeRules Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    737109
           1       0.60      0.58      0.59      3849

    accuracy                           1.00    740958
   macro avg       0.80      0.79      0.79    740958
weighted avg       1.00      1.00      1.00    740958



In [12]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred_skope)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Fraud", "Fraud"])
disp.plot(cmap="Oranges")